In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

# Store F1 Data into a DataFrame

1. Original Data source 1 (FORMAT: CSV): Formula 1 race data - Formula 1 Race Data from 1950 to 2017 https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
    
2. Original Data source 2 (FORMAT: CSV): Weather data is the Local Climatologcial Data from NOAA.gov - https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd

In [2]:
# Read in F1 Grand Prix data from https://www.kaggle.com/cjgdev/formula-1-race-data-19502017
races_csv = pd.read_csv("resources/f1_csvs/races.csv") 
results_csv = pd.read_csv("resources/f1_csvs/results.csv")
circuits_csv = pd.read_csv("resources/f1_csvs/circuits.csv")
drivers_csv = pd.read_csv("resources/f1_csvs/drivers.csv")

# Clean F1 Data

1. Get rid of unknown or uncommon races and only pull US races from circuits.csv and races.csv and then merge them
2. Merge drivers and results csvs on driver id
3. combine the two merged dataframes, drop unwanted columns, rename poorly named columns

In [3]:
# Set the index for the circuits csv to circuitId
circuits_csv = circuits_csv.set_index('circuitId')

# Locate only circuits in the US
circuits_csv = circuits_csv.loc[circuits_csv['country'] == 'USA']

# drop uncommon or unknown US circuits based on circuitId (Index)
circuits_csv = circuits_csv.drop([37, 42, 43, 44, 72])

# Reset the index to make merging easier later
circuits_csv = circuits_csv.reset_index()

circuits_csv.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...
1,33,phoenix,Phoenix street circuit,Phoenix,USA,33.4479,-112.0750,NaN,http://en.wikipedia.org/wiki/Phoenix_street_ci...
2,46,watkins_glen,Watkins Glen,New York State,USA,42.3369,-76.9272,NaN,http://en.wikipedia.org/wiki/Watkins_Glen_Inte...
3,60,riverside,Riverside International Raceway,California,USA,33.9370,-117.2730,NaN,http://en.wikipedia.org/wiki/Riverside_Interna...
4,63,sebring,Sebring International Raceway,Florida,USA,27.4547,-81.3483,NaN,http://en.wikipedia.org/wiki/Sebring_Raceway


In [4]:
# Remove uncommon or unknown US circuits
races_csv = races_csv.loc[races_csv['circuitId'] != 43]

# Save only US Races
races_csv = races_csv.loc[races_csv['name'].isin(['United States Grand Prix', 'United States Grand Prix West'])]
races_csv.head()

,raceId,year,round,circuitId,name,date,time,url
41,42,2007,7,19,United States Grand Prix,6/17/2007,17:00:00,http://en.wikipedia.org/wiki/2007_United_State...
61,62,2006,10,19,United States Grand Prix,7/2/2006,14:00:00,http://en.wikipedia.org/wiki/2006_United_State...
78,79,2005,9,19,United States Grand Prix,6/19/2005,14:00:00,http://en.wikipedia.org/wiki/2005_United_State...
97,98,2004,9,19,United States Grand Prix,6/20/2004,NaN,http://en.wikipedia.org/wiki/2004_United_State...
121,122,2003,15,19,United States Grand Prix,9/28/2003,NaN,http://en.wikipedia.org/wiki/2003_United_State...


In [5]:
# Combine the circuits and races dfs
cir_race = pd.merge(circuits_csv, races_csv, on='circuitId')
cir_race.head()

,circuitId,circuitRef,name_x,location,country,lat,lng,alt,url_x,raceId,year,round,name_y,date,time,url_y
0,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.795,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,42,2007,7,United States Grand Prix,6/17/2007,17:00:00,http://en.wikipedia.org/wiki/2007_United_State...
1,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.795,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,62,2006,10,United States Grand Prix,7/2/2006,14:00:00,http://en.wikipedia.org/wiki/2006_United_State...
2,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.795,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,79,2005,9,United States Grand Prix,6/19/2005,14:00:00,http://en.wikipedia.org/wiki/2005_United_State...
3,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.795,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,98,2004,9,United States Grand Prix,6/20/2004,NaN,http://en.wikipedia.org/wiki/2004_United_State...
4,19,indianapolis,Indianapolis Motor Speedway,Indianapolis,USA,39.795,-86.2347,NaN,http://en.wikipedia.org/wiki/Indianapolis_Moto...,122,2003,15,United States Grand Prix,9/28/2003,NaN,http://en.wikipedia.org/wiki/2003_United_State...


In [6]:
# Combine the drivers and the results dfs
dri_res = pd.merge(drivers_csv, results_csv, on='driverId')
dri_res.head()

,driverId,driverRef,number_x,code,forename,surname,dob,nationality,url,resultId,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,...,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,27,...,5,4.0,56,46.548,5525103.0,53.0,3.0,01:35.5,209.033,1
2,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,57,...,13,0.0,56,NaN,NaN,25.0,19.0,01:35.5,203.969,11
3,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,69,...,3,6.0,66,4.187,5903238.0,20.0,3.0,01:22.0,204.323,1
4,1,hamilton,44.0,HAM,Lewis,Hamilton,7/1/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,90,...,2,8.0,58,3.779,5213230.0,31.0,2.0,01:26.5,222.085,1


In [7]:
# combine the circuits/race df and the driver/results df into one final combined df
combined_df = pd.merge(cir_race, dri_res, on='raceId')
combined_df = combined_df.drop(columns=['circuitRef', 'lat', 'lng', 'alt', 'url_x', 'url_y', 'url', 'name_y', 'time_x', 'round', 'constructorId', 'number_y', 'grid', 'position', 'positionText', 'milliseconds',
       'fastestLap', 'statusId', 'rank', 'resultId', 'number_x', 'driverRef', 'code'])

# Rename columns to something more meaningful
combined_df = combined_df.rename(columns={'name_x':'trackName',
                          'forename':'firstName',
                          'time_y':'driverRaceTime'})
combined_df.head()

,circuitId,trackName,location,country,raceId,year,date,driverId,firstName,surname,dob,nationality,positionOrder,points,laps,driverRaceTime,fastestLapTime,fastestLapSpeed
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,7/1/1985,British,1,10.0,73,31:10.0,01:13.2,206.101
1,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,2,Nick,Heidfeld,10/5/1977,German,18,0.0,56,NaN,01:13.4,205.562
2,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,3,Nico,Rosberg,27/06/1985,German,16,0.0,68,NaN,01:14.1,203.753
3,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,4,Fernando,Alonso,29/07/1981,Spanish,2,8.0,73,1.5,01:13.3,206.003
4,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,5,Heikki,Kovalainen,19/10/1981,Finnish,5,4.0,73,41.4,01:14.0,203.94


# Store Weather Data into a DataFrame

2. Original Data source 2 (FORMAT: NOT YET KNOWN): Weather data is the Local Climatologcial Data from NOAA.gov - https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd

In [8]:
# Create dataframe from csv
austin_weather = pd.read_csv('resources/weather_csvs/austin_weather.csv')
indy_weather = pd.read_csv('resources/weather_csvs/indy_weather.csv')

C:\Users\sengeling\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (33,34,41,42,43,46,49,56,73,75,76,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\sengeling\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31,33,34,41,42,43,46,48,49,51,52,53,56,58,59,60,61,63,69,71,73,75,76,77,82,88,89,95,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Clean Weather Data

In [9]:
# Keep only the rows with code type 'SOD' (indicating daily weather data) in the 'REPORT_TYPE' column
austin_weather = austin_weather.loc[austin_weather['REPORT_TYPE'].str.strip() == 'SOD']

# Drop all columns that have all NaN values
austin_weather.dropna(axis=1, how='all', inplace=True)

# Add column that clarifies the city name
austin_weather['city'] = 'Austin, TX'

# From the remaining columns, keep the ones specified
austin_weather = austin_weather[['STATION', 'DATE', 'city', 'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity', 
                          'DailyAverageStationPressure', 'DailyAverageWindSpeed', 'DailyMaximumDryBulbTemperature', 
                          'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection', 'DailyPrecipitation']]

# Rename columns, reset index
austin_weather.columns = ['station', 'date', 'city', 'avg_temp', 'avg_rel_humidity', 'avg_pressure', 'avg_wind', 'max_temp',
                         'min_temp', 'peak_wind_dir', 'precip']

austin_weather.reset_index(drop=True, inplace=True)

# Clean the 'date' column by keeping only the date ('YYYY-MM-DD') and dropping extra info (time entered)
austin_weather['date'] = [austin_weather['date'][x][:10] for x in range(len(austin_weather['date']))]

# Instantiate list of dates
austin_dates = ['2012-11-18', '2013-11-17', '2014-11-02', '2015-10-25', '2016-10-23', '2017-10-22', '2018-10-21']

# Keep only rows whose date is in the list austin_dates and reset the index again
austin_weather = austin_weather.loc[austin_weather['date'].isin(austin_dates)]

austin_weather.reset_index(drop=True, inplace=True)

# Show dataframe
austin_weather

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72254013904,2012-11-18,"Austin, TX",55.0,65.0,29.70,4.6,74.0,36.0,160.0,0.00
1,72254013904,2013-11-17,"Austin, TX",75.0,78.0,29.33,5.0,89.0,61.0,200.0,0.00
2,72254013904,2014-11-02,"Austin, TX",57.0,64.0,29.68,8.6,76.0,38.0,150.0,0.00
3,72254013904,2015-10-25,"Austin, TX",62.0,88.0,29.47,16.7,65.0,58.0,350.0,0.57
4,72254013904,2016-10-23,"Austin, TX",66.0,68.0,29.62,5.6,84.0,47.0,180.0,0.01
5,72254013904,2017-10-22,"Austin, TX",64.0,81.0,29.57,8.2,78.0,50.0,330.0,0.70
6,72254013904,2018-10-21,"Austin, TX",65.0,61.0,29.70,9.6,74.0,55.0,20.0,0.00


In [10]:
# Keep only the rows with code type 'SOD' (indicating daily weather data) in the 'REPORT_TYPE' column
indy_weather = indy_weather.loc[indy_weather['REPORT_TYPE'].str.strip() == 'SOD']

# Drop all columns that have all NaN values
indy_weather.dropna(axis=1, how='all', inplace=True)

# Add column that clarifies the city name
indy_weather['city'] = 'Indianapolis, IN'

# From the remaining columns, keep the ones specified
indy_weather = indy_weather[['STATION', 'DATE', 'city', 'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity', 
                          'DailyAverageStationPressure', 'DailyAverageWindSpeed', 'DailyMaximumDryBulbTemperature', 
                          'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection', 'DailyPrecipitation']]

# Rename columns, reset index
indy_weather.columns = ['station', 'date', 'city', 'avg_temp', 'avg_rel_humidity', 'avg_pressure', 'avg_wind', 'max_temp',
                         'min_temp', 'peak_wind_dir', 'precip']

indy_weather.reset_index(drop=True, inplace=True)

# Clean the 'date' column by keeping only the date ('YYYY-MM-DD') and dropping extra info (time entered)
indy_weather['date'] = [indy_weather['date'][x][:10] for x in range(len(indy_weather['date']))]

# Instantiate list of dates
indy_dates = ['2000-09-24', '2001-09-30', '2002-09-29', '2003-09-28', '2004-06-20', '2005-06-19', 
              '2006-07-02', '2007-06-17']

# Keep only rows whose date is in the list austin_dates and reset the index again
indy_weather = indy_weather.loc[indy_weather['date'].isin(indy_dates)]

indy_weather.reset_index(drop=True, inplace=True)

# Show dataframe
indy_weather

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,2000-09-24,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,2001-09-30,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,2002-09-29,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,2003-09-28,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,2004-06-20,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s
5,72438093819,2005-06-19,"Indianapolis, IN",67,NaN,29.26,8.1,77,57,040,0.00
6,72438093819,2006-07-02,"Indianapolis, IN",80,63.0,29.21,10.0,90,70,270,0.00
7,72438093819,2007-06-17,"Indianapolis, IN",82,49.0,29.15,7.8,93,71,260,T


# Combine the Weather DataFrames
Commented out as a failsafe.

In [ ]:
# Save dataframe to a csv file
# indy_weather.to_csv('weather_csvs/weather_data.csv', index=False)

In [ ]:
# Append the other dataframe to the weather_data csv
# with open('weather_csvs/weather_data.csv', 'a') as file:
#     austin_weather.to_csv(file, header=False, index=False)

In [11]:
# Create dataframe from the weather_data csv
combined_weather = pd.read_csv('resources/weather_csvs/weather_data.csv')

combined_weather

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,9/24/2000,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,9/30/2001,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,9/29/2002,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,9/28/2003,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,6/20/2004,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s
5,72438093819,6/19/2005,"Indianapolis, IN",67,NaN,29.26,8.1,77,57,40,0
6,72438093819,7/2/2006,"Indianapolis, IN",80,63.0,29.21,10.0,90,70,270,0
7,72438093819,6/17/2007,"Indianapolis, IN",82,49.0,29.15,7.8,93,71,260,T
8,72254013904,11/18/2012,"Austin, TX",55,65.0,29.70,4.6,74,36,160,0
9,72254013904,11/17/2013,"Austin, TX",75,78.0,29.33,5.0,89,61,200,0


# Create Database
Done in Stephanie's Workbench - f1_weather_db.

# Connect to Local

Relational database, chosen in order to join two different types of data on a common key - date

In [12]:
# Connect to the database
rds_connection_string = "root:Stup!dR00tP@55w0rd@127.0.0.1/f1_weather_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Use Pandas to Load Race Data into Database as a Table
f1_data


In [13]:
# Add the combined f1 dataframe to the database
combined_df.to_sql(name='f1_data', con=engine, if_exists='append', index=False)

# Use Pandas to Load Weather Data into Database as a Table
weather_data

In [14]:
combined_weather.to_sql(name='weather_data', con=engine, if_exists='append', index=False)

# Confirm Data Has Been Loaded

In [15]:
# Test that the f1 table is presnt and working
pd.read_sql_query('select * from f1_data', con=engine).head()

,circuitId,trackName,location,country,raceId,year,date,driverId,firstName,surname,dob,nationality,positionOrder,points,laps,driverRaceTime,fastestLapTime,fastestLapSpeed
0,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,1,Lewis,Hamilton,7/1/1985,British,1,10.0,73,31:10.0,01:13.2,206.101
1,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,2,Nick,Heidfeld,10/5/1977,German,18,0.0,56,None,01:13.4,205.562
2,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,3,Nico,Rosberg,27/06/1985,German,16,0.0,68,None,01:14.1,203.753
3,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,4,Fernando,Alonso,29/07/1981,Spanish,2,8.0,73,1.5,01:13.3,206.003
4,19,Indianapolis Motor Speedway,Indianapolis,USA,42,2007,6/17/2007,5,Heikki,Kovalainen,19/10/1981,Finnish,5,4.0,73,41.4,01:14.0,203.94


In [16]:
# Test that the weather table is presnt and working
pd.read_sql_query('select * from weather_data', con=engine).head()

,station,date,city,avg_temp,avg_rel_humidity,avg_pressure,avg_wind,max_temp,min_temp,peak_wind_dir,precip
0,72438093819,9/24/2000,"Indianapolis, IN",59,NaN,29.08,10.9,63,55,330,0.23
1,72438093819,9/30/2001,"Indianapolis, IN",59,NaN,29.34,6.8,72,46,360,0.00s
2,72438093819,9/29/2002,"Indianapolis, IN",69,NaN,29.24,6.8,80,57,190,0.00s
3,72438093819,9/28/2003,"Indianapolis, IN",50,NaN,29.05,8.9,56,44,320,0.07
4,72438093819,6/20/2004,"Indianapolis, IN",63,NaN,29.21,6.0,75,51,250,0.00s


# Test a Join on Date

In [22]:
pd.read_sql_query('select f1_data.location, f1_data.date, f1_data.fastestLapTime, f1_data.fastestLapSpeed, weather_data.avg_temp, weather_data.avg_rel_humidity, weather_data.avg_wind from f1_data inner join weather_data on f1_data.date = weather_data.date', con=engine)


,location,date,fastestLapTime,fastestLapSpeed,avg_temp,avg_rel_humidity,avg_wind
0,Indianapolis,6/17/2007,01:13.2,206.101,82,49.0,7.8
1,Indianapolis,6/17/2007,01:13.2,206.101,82,49.0,7.8
2,Indianapolis,6/17/2007,01:13.2,206.101,82,49.0,7.8
3,Indianapolis,6/17/2007,01:13.4,205.562,82,49.0,7.8
4,Indianapolis,6/17/2007,01:13.4,205.562,82,49.0,7.8
5,Indianapolis,6/17/2007,01:13.4,205.562,82,49.0,7.8
6,Indianapolis,6/17/2007,01:14.1,203.753,82,49.0,7.8
7,Indianapolis,6/17/2007,01:14.1,203.753,82,49.0,7.8
8,Indianapolis,6/17/2007,01:14.1,203.753,82,49.0,7.8
9,Indianapolis,6/17/2007,01:13.3,206.003,82,49.0,7.8
